In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../Datasets/LTC.csv")

In [3]:
df.head()

,Crypto,Date,Open,High,Low,Close
0,LTC,12/31/22,67.78,71.04,67.56,70.29
1,LTC,12/30/22,66.08,68.57,66.04,67.78
2,LTC,12/29/22,66.39,67.16,65.58,66.08
3,LTC,12/28/22,68.08,69.37,66.09,66.42
4,LTC,12/27/22,69.92,71.03,67.80,68.08


In [4]:
print("Dataframe Shape: " , df.shape)
print("Dataframe feature Datatypes: ", df.dtypes)
print("Dataframe Description: ", df.describe())

Dataframe Shape:  (1975, 6)
Dataframe feature Datatypes:  Crypto     object
Date       object
Open      float64
High      float64
Low       float64
Close     float64
dtype: object
Dataframe Description:                Open         High          Low        Close
count  1975.000000  1975.000000  1975.000000  1975.000000
mean     98.426248   102.451261    94.191747    98.345600
std      58.750386    62.229616    55.228690    58.674749
min      22.660000    23.470000    22.260000    22.580000
25%      54.745000    56.390000    52.745000    54.765000
50%      78.410000    81.680000    75.600000    78.190000
75%     132.380000   137.100000   125.935000   132.130000
max     373.350000   413.220000   345.760000   373.640000


In [5]:
df.isnull().sum()

Crypto    0
Date      0
Open      0
High      0
Low       0
Close     0
dtype: int64